<h1>Next Word Prediction Model Using Tensorflow & keras<h1> 

<h5>Importing Libraries<h5>

In [1]:
from numpy import array
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import load_model


<h5>Preprocessing Data<h5>

In [2]:
class Preprocessing():
    
    def __init__(self,input_file):
        self.input_data_file = input_file
        self.data = None
        self.vocab_size = None
        self.encoded_data = None
        self.max_length = None
        self.sequences = None
        self.x = None
        self.y = None
        self.tokenizer = None
    
    def load_data(self):
        fp = open(self.input_data_file,'r')
        self.data = fp.read().splitlines()        
        fp.close()
        
    def encode_data(self):
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.data)
        self.encoded_data = self.tokenizer.texts_to_sequences(self.data)
        self.vocab_size = len(self.tokenizer.word_counts)+1
        
    def generate_sequence(self):
        seq_list = list()
        for item in self.encoded_data:
            l = len(item)
            for id in range(1,l):
                seq_list.append(item[:id+1])
        self.max_length = max([len(seq) for seq in seq_list])
        self.sequences = pad_sequences(seq_list, maxlen=self.max_length, padding='pre')
        self.sequences = array(self.sequences)
            
    def get_data(self):
        self.x = self.sequences[:,:-1]
        self.y = self.sequences[:,-1]
        self.y = to_categorical(self.y,num_classes=self.vocab_size)

In [3]:
pr = Preprocessing('cab_booking.txt')
pr.load_data()
pr.encode_data()
pr.generate_sequence()
pr.get_data()

<h3>Model

In [4]:
class Model():
    def __init__(self):
        self.model = None
        self.history = None
        self.x = None
        self.y = None
        self.vocab_size = pr.vocab_size
        self.max_len = pr.max_length
        
        
    def create_model(self):
        self.model = Sequential()
        self.model.add(Embedding(self.vocab_size,10,input_length=self.max_len-1))
        self.model.add(LSTM(50))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(self.vocab_size,activation='softmax'))
        self.model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        print(self.model.summary())
    def run(self,epochs,batch_size):
        self.history = self.model.fit(self.x,self.y,epochs=epochs,batch_size=batch_size,validation_split=0.2)
        
    def save(self):
        self.model.save("word_prediction_model.h5")
        

In [5]:
model_obj = Model()
model_obj.x = pr.x
model_obj.y = pr.y
model_obj.create_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 10)             340       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 34)                1734      
Total params: 14,274
Trainable params: 14,274
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model_obj.run(700,2)
model_obj.save()

Train on 59 samples, validate on 15 samples
Epoch 1/700
59/59 [==============================] - 2s 36ms/sample - loss: 3.5235 - accuracy: 0.0169 - val_loss: 3.5263 - val_accuracy: 0.0667
Epoch 2/700
59/59 [==============================] - 0s 3ms/sample - loss: 3.4933 - accuracy: 0.0847 - val_loss: 3.5263 - val_accuracy: 0.0000e+00
Epoch 3/700
59/59 [==============================] - 0s 3ms/sample - loss: 3.4002 - accuracy: 0.0847 - val_loss: 3.5981 - val_accuracy: 0.0667
Epoch 4/700
59/59 [==============================] - 0s 3ms/sample - loss: 3.2136 - accuracy: 0.0508 - val_loss: 3.8809 - val_accuracy: 0.0000e+00
Epoch 5/700
59/59 [==============================] - 0s 3ms/sample - loss: 3.0548 - accuracy: 0.0847 - val_loss: 3.9181 - val_accuracy: 0.0000e+00
Epoch 6/700
59/59 [==============================] - 0s 4ms/sample - loss: 2.9402 - accuracy: 0.0847 - val_loss: 4.0749 - val_accuracy: 0.0000e+00
Epoch 7/700
59/59 [==============================] - 0s 3ms/sample - loss: 2.8204

59/59 [==============================] - 0s 4ms/sample - loss: 0.7604 - accuracy: 0.8475 - val_loss: 4.5184 - val_accuracy: 0.3333
Epoch 58/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.7599 - accuracy: 0.8814 - val_loss: 4.5606 - val_accuracy: 0.2667
Epoch 59/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.8139 - accuracy: 0.8305 - val_loss: 4.6590 - val_accuracy: 0.2667
Epoch 60/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.7477 - accuracy: 0.7966 - val_loss: 4.5774 - val_accuracy: 0.2667
Epoch 61/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.7172 - accuracy: 0.8305 - val_loss: 4.6563 - val_accuracy: 0.3333
Epoch 62/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.6767 - accuracy: 0.8475 - val_loss: 4.6746 - val_accuracy: 0.3333
Epoch 63/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.6861 - accuracy: 0.8305 - val_loss: 4.7877 - val_accuracy: 0.2000
Epoch

Epoch 114/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.3242 - accuracy: 0.8814 - val_loss: 4.8929 - val_accuracy: 0.3333
Epoch 115/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.3244 - accuracy: 0.9153 - val_loss: 4.9357 - val_accuracy: 0.2667
Epoch 116/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.3149 - accuracy: 0.8983 - val_loss: 4.7738 - val_accuracy: 0.3333
Epoch 117/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.3164 - accuracy: 0.8983 - val_loss: 4.8997 - val_accuracy: 0.3333
Epoch 118/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.3333 - accuracy: 0.8814 - val_loss: 4.7328 - val_accuracy: 0.3333
Epoch 119/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.3225 - accuracy: 0.8814 - val_loss: 4.7383 - val_accuracy: 0.3333
Epoch 120/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2970 - accuracy: 0.8983 - val_loss: 4.7394 - val_ac

59/59 [==============================] - 0s 5ms/sample - loss: 0.2457 - accuracy: 0.8983 - val_loss: 4.7832 - val_accuracy: 0.3333
Epoch 171/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2368 - accuracy: 0.8983 - val_loss: 4.7573 - val_accuracy: 0.3333
Epoch 172/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2528 - accuracy: 0.8644 - val_loss: 4.7994 - val_accuracy: 0.3333
Epoch 173/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2600 - accuracy: 0.8644 - val_loss: 4.7274 - val_accuracy: 0.3333
Epoch 174/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2534 - accuracy: 0.9153 - val_loss: 4.6931 - val_accuracy: 0.3333
Epoch 175/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2416 - accuracy: 0.8983 - val_loss: 4.7300 - val_accuracy: 0.3333
Epoch 176/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2565 - accuracy: 0.8814 - val_loss: 4.8124 - val_accuracy: 0.2667

59/59 [==============================] - 0s 4ms/sample - loss: 0.2113 - accuracy: 0.9153 - val_loss: 4.7536 - val_accuracy: 0.3333
Epoch 227/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1917 - accuracy: 0.9153 - val_loss: 4.6992 - val_accuracy: 0.3333
Epoch 228/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2193 - accuracy: 0.8983 - val_loss: 4.7147 - val_accuracy: 0.3333
Epoch 229/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2111 - accuracy: 0.9153 - val_loss: 4.6245 - val_accuracy: 0.3333
Epoch 230/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2282 - accuracy: 0.8814 - val_loss: 4.6499 - val_accuracy: 0.3333
Epoch 231/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2073 - accuracy: 0.8814 - val_loss: 4.6528 - val_accuracy: 0.3333
Epoch 232/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2108 - accuracy: 0.8983 - val_loss: 4.6252 - val_accuracy: 0.2667

...

59/59 [==============================] - 0s 3ms/sample - loss: 0.1987 - accuracy: 0.8814 - val_loss: 4.3604 - val_accuracy: 0.2667
Epoch 451/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1748 - accuracy: 0.9322 - val_loss: 4.3631 - val_accuracy: 0.2667
Epoch 452/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2079 - accuracy: 0.8983 - val_loss: 4.3245 - val_accuracy: 0.3333
Epoch 453/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2020 - accuracy: 0.8644 - val_loss: 4.4464 - val_accuracy: 0.2667
Epoch 454/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1851 - accuracy: 0.9153 - val_loss: 4.4501 - val_accuracy: 0.2667
Epoch 455/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2107 - accuracy: 0.8644 - val_loss: 4.3835 - val_accuracy: 0.2667
Epoch 456/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2078 - accuracy: 0.8644 - val_loss: 4.3403 - val_accuracy: 0.2667

59/59 [==============================] - 0s 3ms/sample - loss: 0.2020 - accuracy: 0.8644 - val_loss: 4.7956 - val_accuracy: 0.2000
Epoch 507/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1827 - accuracy: 0.8983 - val_loss: 4.7852 - val_accuracy: 0.1333
Epoch 508/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2109 - accuracy: 0.8983 - val_loss: 4.7720 - val_accuracy: 0.2000
Epoch 509/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2169 - accuracy: 0.8644 - val_loss: 4.7427 - val_accuracy: 0.2000
Epoch 510/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1905 - accuracy: 0.9153 - val_loss: 4.7872 - val_accuracy: 0.1333
Epoch 511/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2297 - accuracy: 0.8814 - val_loss: 4.7726 - val_accuracy: 0.1333
Epoch 512/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2051 - accuracy: 0.8983 - val_loss: 4.7755 - val_accuracy: 0.2000

59/59 [==============================] - 0s 4ms/sample - loss: 0.2150 - accuracy: 0.8814 - val_loss: 4.7893 - val_accuracy: 0.2000
Epoch 563/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1889 - accuracy: 0.8644 - val_loss: 4.8024 - val_accuracy: 0.2667
Epoch 564/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1916 - accuracy: 0.9153 - val_loss: 4.7838 - val_accuracy: 0.2667
Epoch 565/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1949 - accuracy: 0.8814 - val_loss: 4.7529 - val_accuracy: 0.2000
Epoch 566/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.2005 - accuracy: 0.9153 - val_loss: 4.7509 - val_accuracy: 0.2667
Epoch 567/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1922 - accuracy: 0.8983 - val_loss: 4.7740 - val_accuracy: 0.2667
Epoch 568/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1870 - accuracy: 0.8814 - val_loss: 4.7780 - val_accuracy: 0.2000

59/59 [==============================] - 0s 3ms/sample - loss: 0.1757 - accuracy: 0.9153 - val_loss: 4.9312 - val_accuracy: 0.2667
Epoch 619/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1866 - accuracy: 0.9322 - val_loss: 4.8547 - val_accuracy: 0.2667
Epoch 620/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.2033 - accuracy: 0.8644 - val_loss: 4.8353 - val_accuracy: 0.2667
Epoch 621/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1994 - accuracy: 0.8814 - val_loss: 4.8321 - val_accuracy: 0.2667
Epoch 622/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1835 - accuracy: 0.9153 - val_loss: 4.8338 - val_accuracy: 0.2667
Epoch 623/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1963 - accuracy: 0.8814 - val_loss: 4.9103 - val_accuracy: 0.2000
Epoch 624/700
59/59 [==============================] - 0s 3ms/sample - loss: 0.1954 - accuracy: 0.8983 - val_loss: 4.8946 - val_accuracy: 0.2667

59/59 [==============================] - 0s 4ms/sample - loss: 0.2000 - accuracy: 0.8644 - val_loss: 5.0174 - val_accuracy: 0.2000
Epoch 675/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1878 - accuracy: 0.8814 - val_loss: 4.8063 - val_accuracy: 0.2667
Epoch 676/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1859 - accuracy: 0.9153 - val_loss: 4.7982 - val_accuracy: 0.2667
Epoch 677/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1823 - accuracy: 0.9153 - val_loss: 4.7930 - val_accuracy: 0.2667
Epoch 678/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1828 - accuracy: 0.8983 - val_loss: 4.7686 - val_accuracy: 0.2667
Epoch 679/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1940 - accuracy: 0.9153 - val_loss: 4.8213 - val_accuracy: 0.2000
Epoch 680/700
59/59 [==============================] - 0s 4ms/sample - loss: 0.1935 - accuracy: 0.8983 - val_loss: 4.8946 - val_accuracy: 0.2000

<h4>Prediction

In [7]:
class Prediction():
    def __init__(self,tokenizer,max_len):
        self.model = None
        self.tokenizer = tokenizer
        self.idx2word = {v:k for k,v in self.tokenizer.word_index.items()}
        self.max_length = max_len
    
    def load_model(self):
        self.model = load_model("word_prediction_model.h5")
        
    def predict_words(self,text,num_words):
        encoded_data = self.tokenizer.texts_to_sequences([text])[0]
        padded_data = pad_sequences([encoded_data],maxlen = self.max_length-1,padding='pre')
        y_preds = self.model.predict(padded_data)
        y_preds = np.argsort(-y_preds)
        y_preds = y_preds[0][:num_words]
        possible_words = [self.idx2word[item] for item in y_preds]
        print(text,possible_words)
        print(possible_words)


In [8]:
pred = Prediction(pr.tokenizer,pr.max_length)    
pred.load_model()

In [9]:
pred.predict_words("I would like to",2)
pred.predict_words("can you please",2)

I would like to ['book', 'cancel']
['book', 'cancel']
can you please ['book', 'cancel']
['book', 'cancel']
